In [3]:
# Imports
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from scipy.sparse import csr_matrix
import string
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import time

In [4]:
# Import test.dat and train.dat
trainData = []
with open('train.dat', 'r') as file:
    for line in file.readlines():
            trainData.append(line[2:])
testData = open('test.dat', 'r').readlines()
fullData = trainData + testData

In [5]:
# Combine datasets for countvectorizer
start = time.time()

cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
count_occurences = cv.fit_transform(fullData)
train_occurences = count_occurences[:len(trainData)]
test_occurences = count_occurences[len(trainData):]

print(time.time() - start)

3.9955711364746094


In [6]:
# Creating train vector dataframe
train_vect_df = pd.DataFrame(data = train_occurences.toarray(), columns= cv.get_feature_names_out())
train_vect_df

MemoryError: Unable to allocate 50.6 GiB for an array with shape (102080, 66494) and data type int64

In [7]:
# Creating test vector dataframe
test_vect_df = pd.DataFrame(data = test_occurences.toarray(), columns= cv.get_feature_names_out())
test_vect_df

,0,00,000,000016447,00010563n,0002,0004287,0005173miami050,000660,0009875buffalo040,...,zwick,zwiki,zy,zydrunas,zygmunt,zyman,zyprexa,zz,zzz,zzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25515,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def cmer(name, c=3):
    r""" Given a string and parameter c, return the vector of k-mers associated with the words
    """
    name = name.lower()
    if len(name) < c:
        return [name]
    v = []
    for i in range(len(name)-c+1):
        v.append(name[i:(i+c)])
    return v

def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [9]:
# Manual method
def textToMatrix(names):
    docs = [n.split() for n in names]
    return build_matrix(docs)

start = time.time()

# Use CSR functions
full_matrix = textToMatrix(fullData)
train_csr = full_matrix[:len(trainData)]
test_csr = full_matrix[len(trainData):]
csr_info(train_csr)
csr_info(test_csr)

print(time.time() - start)

 [nrows 102080, ncols 194175, nnz 3396479]
 [nrows 25520, ncols 194175, nnz 849425]
5.463740348815918
